In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
import os

In [ ]:
model=ChatOpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=os.getenv("OPENROUTER_DEEPSEEK_API_KEY"),  # or hardcode for now
            model="deepseek/deepseek-chat",
            temperature=0.7,
            max_tokens=500
        )

In [ ]:
from langgraph.graph import add_messages #is a reducer similar to add  but for 
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], "List of chat messages",add_messages]
    user_id: Annotated[str, "Unique identifier for the user"]
    chat_id: Annotated[str, "Unique identifier for the chat session"]
    context: Annotated[dict[str, str], "Contextual information for the chat"]

In [ ]:
def chat_node(state: ChatState):
    messages=state['messages']
    response=model.invoke(messages)
    return {'messages': {response}}

In [ ]:
checkpointer=MemorySaver()
graph=StateGraph(ChatState)
graph.add_node(chat_node,'chat_node')

graph.add_edge(START, 'chat_node')
graph.add_edge('chat_node', END)

chatbot= graph.compile(checkpointer=checkpointer)

In [ ]:
initial_state={
    'messages': [HumanMessage(content="Hello")],
    'user_id': 'user-123',
    'chat_id': 'chat-456',
    'context': {}
}

chatbot.invoke(initial_state)

In [ ]:
thread_id='1'
while True:
    user_message=input("Your message here: ")
    print('You: ', user_message)
    if user_message.lower() in ['exit', 'quit']:
        break
    config={'configurable':{'thread_id':thread_id}}
    response=chatbot.invoke({'messages':[HumanMessage(content=user_message)]},config=config)
    print("Chatbot response:", response['messages'][-1].content)
   # Assuming the last message is the bot's response

In [ ]:
chatbot.get_state(config=config)